In [1]:
# Cell 1: compute BLOSUM62, hydropathy, is_stop, save stage2
import os, pandas as pd, numpy as np

proc_dir = r"C:\BLAST\variant-effect-prediction\data\processed"
in_path = os.path.join(proc_dir, "clinvar_subset_parsed_aa.csv")  # produced earlier
out_stage2 = os.path.join(proc_dir, "clinvar_features_stage2_full.csv")

df = pd.read_csv(in_path, low_memory=False)
print("Loaded rows:", len(df))

# ---------- BLOSUM62 table (complete) ----------
_blosum = {
'A':{'A':4,'R':-1,'N':-2,'D':-2,'C':0,'Q':-1,'E':-1,'G':0,'H':-2,'I':-1,'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S':1,'T':0,'W':-3,'Y':-2,'V':0},
'R':{'A':-1,'R':5,'N':0,'D':-2,'C':-3,'Q':1,'E':0,'G':-2,'H':0,'I':-3,'L':-2,'K':2,'M':-1,'F':-3,'P':-2,'S':-1,'T':-1,'W':-3,'Y':-2,'V':-3},
'N':{'A':-2,'R':0,'N':6,'D':1,'C':-3,'Q':0,'E':0,'G':0,'H':1,'I':-3,'L':-3,'K':0,'M':-2,'F':-3,'P':-2,'S':1,'T':0,'W':-4,'Y':-2,'V':-3},
'D':{'A':-2,'R':-2,'N':1,'D':6,'C':-3,'Q':0,'E':2,'G':-1,'H':-1,'I':-3,'L':-4,'K':-1,'M':-3,'F':-3,'P':-1,'S':0,'T':-1,'W':-4,'Y':-3,'V':-3},
'C':{'A':0,'R':-3,'N':-3,'D':-3,'C':9,'Q':-3,'E':-4,'G':-3,'H':-3,'I':-1,'L':-1,'K':-3,'M':-1,'F':-2,'P':-3,'S':-1,'T':-1,'W':-2,'Y':-2,'V':-1},
'Q':{'A':-1,'R':1,'N':0,'D':0,'C':-3,'Q':5,'E':2,'G':-2,'H':0,'I':-3,'L':-2,'K':1,'M':0,'F':-3,'P':-1,'S':0,'T':-1,'W':-2,'Y':-1,'V':-2},
'E':{'A':-1,'R':0,'N':0,'D':2,'C':-4,'Q':2,'E':5,'G':-2,'H':0,'I':-3,'L':-3,'K':1,'M':-2,'F':-3,'P':-1,'S':0,'T':-1,'W':-3,'Y':-2,'V':-2},
'G':{'A':0,'R':-2,'N':0,'D':-1,'C':-3,'Q':-2,'E':-2,'G':6,'H':-2,'I':-4,'L':-4,'K':-2,'M':-3,'F':-3,'P':-2,'S':0,'T':-2,'W':-2,'Y':-3,'V':-3},
'H':{'A':-2,'R':0,'N':1,'D':-1,'C':-3,'Q':0,'E':0,'G':-2,'H':8,'I':-3,'L':-3,'K':-1,'M':-2,'F':-1,'P':-2,'S':-1,'T':-2,'W':-2,'Y':2,'V':-3},
'I':{'A':-1,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-3,'E':-3,'G':-4,'H':-3,'I':4,'L':2,'K':-3,'M':1,'F':0,'P':-3,'S':-2,'T':-1,'W':-3,'Y':-1,'V':3},
'L':{'A':-1,'R':-2,'N':-3,'D':-4,'C':-1,'Q':-2,'E':-3,'G':-4,'H':-3,'I':2,'L':4,'K':-2,'M':2,'F':0,'P':-3,'S':-2,'T':-1,'W':-2,'Y':-1,'V':1},
'K':{'A':-1,'R':2,'N':0,'D':-1,'C':-3,'Q':1,'E':1,'G':-2,'H':-1,'I':-3,'L':-2,'K':5,'M':-1,'F':-3,'P':-1,'S':0,'T':-1,'W':-3,'Y':-2,'V':-2},
'M':{'A':-1,'R':-1,'N':-2,'D':-3,'C':-1,'Q':0,'E':-2,'G':-3,'H':-2,'I':1,'L':2,'K':-1,'M':5,'F':0,'P':-2,'S':-1,'T':-1,'W':-1,'Y':-1,'V':1},
'F':{'A':-2,'R':-3,'N':-3,'D':-3,'C':-2,'Q':-3,'E':-3,'G':-3,'H':-1,'I':0,'L':0,'K':-3,'M':0,'F':6,'P':-4,'S':-2,'T':-2,'W':1,'Y':3,'V':-1},
'P':{'A':-1,'R':-2,'N':-2,'D':-1,'C':-3,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-3,'L':-3,'K':-1,'M':-2,'F':-4,'P':7,'S':-1,'T':-1,'W':-4,'Y':-3,'V':-2},
'S':{'A':1,'R':-1,'N':1,'D':0,'C':-1,'Q':0,'E':0,'G':0,'H':-1,'I':-2,'L':-2,'K':0,'M':-1,'F':-2,'P':-1,'S':4,'T':1,'W':-3,'Y':-2,'V':-2},
'T':{'A':0,'R':-1,'N':0,'D':-1,'C':-1,'Q':-1,'E':-1,'G':-2,'H':-2,'I':-1,'L':-1,'K':-1,'M':-1,'F':-2,'P':-1,'S':1,'T':5,'W':-2,'Y':-2,'V':0},
'W':{'A':-3,'R':-3,'N':-4,'D':-4,'C':-2,'Q':-2,'E':-3,'G':-2,'H':-2,'I':-3,'L':-2,'K':-3,'M':-1,'F':1,'P':-4,'S':-3,'T':-2,'W':11,'Y':2,'V':-3},
'Y':{'A':-2,'R':-2,'N':-2,'D':-3,'C':-2,'Q':-1,'E':-2,'G':-3,'H':2,'I':-1,'L':-1,'K':-2,'M':-1,'F':3,'P':-3,'S':-2,'T':-2,'W':2,'Y':7,'V':-1},
'V':{'A':0,'R':-3,'N':-3,'D':-3,'C':-1,'Q':-2,'E':-2,'G':-3,'H':-3,'I':3,'L':1,'K':-2,'M':1,'F':-1,'P':-2,'S':-2,'T':0,'W':-3,'Y':-1,'V':4}
}

# hydropathy (Kyte-Doolittle)
_hydro = {
 'A':1.8,'R':-4.5,'N':-3.5,'D':-3.5,'C':2.5,'Q':-3.5,'E':-3.5,'G':-0.4,'H':-3.2,
 'I':4.5,'L':3.8,'K':-3.9,'M':1.9,'F':2.8,'P':-1.6,'S':-0.8,'T':-0.7,'W':-0.9,'Y':-1.3,'V':4.2,'*':np.nan
}

def blosum_score(a,b):
    if pd.isna(a) or pd.isna(b): return np.nan
    a=str(a).upper(); b=str(b).upper()
    if a not in _blosum or b not in _blosum[a]:
        return np.nan
    return _blosum[a].get(b, np.nan)

def hydro_diff(a,b):
    if pd.isna(a) or pd.isna(b): return np.nan
    a=str(a).upper(); b=str(b).upper()
    if a not in _hydro or b not in _hydro: return np.nan
    if np.isnan(_hydro[a]) or np.isnan(_hydro[b]): return np.nan
    return abs(_hydro[a] - _hydro[b])

# compute features
df["blosum62_raw"] = df.apply(lambda r: blosum_score(r.get("ref_aa"), r.get("alt_aa")), axis=1)
df["hydropathy_diff"] = df.apply(lambda r: hydro_diff(r.get("ref_aa"), r.get("alt_aa")), axis=1)
df["is_stop"] = df["alt_aa"].apply(lambda x: 1 if str(x) == "*" else 0)

# quick stats
print("Non-null counts:")
print(df[["ref_aa","alt_aa","blosum62_raw","hydropathy_diff","is_stop"]].notnull().sum())

# Save stage2
df.to_csv(out_stage2, index=False)
print("Saved stage2 to:", out_stage2)


Loaded rows: 20000
Non-null counts:
ref_aa              5175
alt_aa              5098
blosum62_raw        2722
hydropathy_diff     2722
is_stop            20000
dtype: int64
Saved stage2 to: C:\BLAST\variant-effect-prediction\data\processed\clinvar_features_stage2_full.csv


In [2]:
# Cell 2: impute, encode, scale -> clinvar_ml_ready.csv
import os, pandas as pd, numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder

proc_dir = r"C:\BLAST\variant-effect-prediction\data\processed"
in_stage2 = os.path.join(proc_dir, "clinvar_features_stage2_full.csv")
out_ml = os.path.join(proc_dir, "clinvar_ml_ready.csv")

df = pd.read_csv(in_stage2, low_memory=False)

# Ensure label numeric 0/1
df["label"] = df["label"].astype(str).str.lower().apply(lambda x: 1 if "pathogenic" in x else (0 if "benign" in x else np.nan))
print("Label counts:", df["label"].value_counts(dropna=False).to_dict())

# numeric features we want
num_feats = ["blosum62_raw","hydropathy_diff","is_stop"]
# allele_freq may be missing; keep if exists
if "allele_freq" in df.columns:
    num_feats.append("allele_freq")
else:
    df["allele_freq"] = np.nan
    num_feats.append("allele_freq")

# fill numeric NaN with medians
for c in num_feats:
    med = df[c].median(skipna=True)
    if np.isnan(med):
        med = 0.0
    df[c] = df[c].fillna(med)
    print(f"Filled {c} NA with median {med}")

# categorical: top genes
if "gene" in df.columns:
    topN = 30
    top_genes = df["gene"].value_counts().nlargest(topN).index.tolist()
    df["gene_top"] = df["gene"].apply(lambda g: g if g in top_genes else "OTHER")
    cat_cols = ["gene_top"]
else:
    cat_cols = []

# One-hot encode
if cat_cols:
    try:
        ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    except TypeError:
        ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
    ohe_mat = ohe.fit_transform(df[cat_cols])
    ohe_df = pd.DataFrame(ohe_mat, columns=ohe.get_feature_names_out(cat_cols), index=df.index)
else:
    ohe_df = pd.DataFrame(index=df.index)

# scale numeric
scaler = StandardScaler()
scaled = scaler.fit_transform(df[num_feats])
scaled_df = pd.DataFrame(scaled, columns=[f"{c}_z" for c in num_feats], index=df.index)

ml_df = pd.concat([df[["label"]].reset_index(drop=True), scaled_df.reset_index(drop=True), ohe_df.reset_index(drop=True)], axis=1)
print("ML-ready shape:", ml_df.shape)

ml_df.to_csv(out_ml, index=False)
print("Saved ML-ready to:", out_ml)


Label counts: {1: 10000, 0: 10000}
Filled blosum62_raw NA with median -1.0
Filled hydropathy_diff NA with median 2.5
Filled is_stop NA with median 0.0
Filled allele_freq NA with median 0.0
ML-ready shape: (20000, 36)


C:\BLAST\envs\varpred\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Saved ML-ready to: C:\BLAST\variant-effect-prediction\data\processed\clinvar_ml_ready.csv


In [3]:
# Week 4 - Cell 3: Baseline ML Models
import pandas as pd, os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

proc_dir = r"C:\BLAST\variant-effect-prediction\data\processed"
ml_path = os.path.join(proc_dir, "clinvar_ml_ready.csv")

df = pd.read_csv(ml_path)
print("ML dataset shape:", df.shape)
print(df.head())

# Separate features/labels
X = df.drop(columns=["label"])
y = df["label"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
print("Train/test split:", X_train.shape, X_test.shape)

# ---- Random Forest ----
rf = RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42, n_jobs=2, class_weight="balanced")
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)[:,1]

print("\nRandom Forest Performance:")
print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print("Precision:", round(precision_score(y_test, y_pred), 3))
print("Recall:", round(recall_score(y_test, y_pred), 3))
print("F1:", round(f1_score(y_test, y_pred), 3))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob), 3))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

# ---- Feature Importance ----
importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop 10 Features:\n", importances.head(10))

# ---- Logistic Regression ----
lr = LogisticRegression(max_iter=2000, class_weight="balanced")
lr.fit(X_train, y_train)
y_prob_lr = lr.predict_proba(X_test)[:,1]
print("\nLogistic Regression ROC-AUC:", round(roc_auc_score(y_test, y_prob_lr), 3))


ML dataset shape: (20000, 36)
   label  blosum62_raw_z  hydropathy_diff_z  is_stop_z  allele_freq_z  \
0      1       -0.097789          -0.024216  -0.366735            0.0   
1      1       -0.097789          -0.024216  -0.366735            0.0   
2      1       -0.097789          -0.024216   2.726767            0.0   
3      1       -0.097789          -0.024216   2.726767            0.0   
4      1       -0.097789          -0.024216  -0.366735            0.0   

   gene_top_-  gene_top_ABCA4  gene_top_APC  gene_top_ATM  gene_top_BRCA1  \
0         0.0             0.0           0.0           0.0             0.0   
1         0.0             0.0           0.0           0.0             0.0   
2         0.0             0.0           0.0           0.0             0.0   
3         0.0             0.0           0.0           0.0             0.0   
4         0.0             0.0           0.0           0.0             0.0   

   ...  gene_top_NF1  gene_top_OTHER  gene_top_PALB2  gene_top_PHEX 

In [4]:
import joblib
model_path = os.path.join(proc_dir, "rf_baseline_model.pkl")
joblib.dump(rf, model_path)
print("Saved RandomForest model to:", model_path)


Saved RandomForest model to: C:\BLAST\variant-effect-prediction\data\processed\rf_baseline_model.pkl
